# Subsections Issue

## Setup

### Imports

In [1]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

In [2]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Two Line SubSection Tests')
logger.setLevel(logging.DEBUG)
#logger.setLevel(logging.INFO)

### Display Functions

In [3]:
#%%writefile display_functions.py

In [4]:
# %% Function to compare context for two sections.
def compare_context(section1, section2):
    ctx_template = '{key:16s}:\t{item1:16s}\t{item2:16s}'
    context_1 = section1.context
    context_2 = section2.context
    keys_1 = set(context_1.keys())
    keys_2 = set(context_2.keys())
    all_keys = keys_1 | keys_2
    for key in all_keys:
        item1 = context_1.get(key, '')
        item2 = context_2.get(key, '')
        ctx_str = ctx_template.format(key=str(key), item1=str(item1), item2=str(item2))
        print(ctx_str)
        
        
# %% Compare Buffered Iterator contents
def buffered_iterator_compare(iter1, iter2=None, iter3=None, 
                              label1='From Iterator', 
                              label2='To Iterator', label3=''):
    
    def extract_attrs(buf_obj, requested_item, as_list=True):
        if not buf_obj:
            text = ''
        elif as_list:
            text = str(list(buf_obj.__getattribute__(requested_item)))
        else:
            text = str(buf_obj.__getattribute__(requested_item))
        return text
        
    def extract_attr_text(requested_item, iter1, iter2=None, iter3=None, 
                        as_list=True):    
        attr_text = {
            1: extract_attrs(iter1, requested_item, as_list),
            2: extract_attrs(iter2, requested_item, as_list),
            3: extract_attrs(iter3, requested_item, as_list),
        }
        return attr_text


    row_template = ''.join([
        '\t{Label:<20s}',
        '{first_iter_item:<35s}',
        '{second_iter_item:<35s}',
        '{third_iter_item:<35s}\n'
        ])   
    attr_group = {
        'Previous Items': ('previous_items', True),
        'Future Items': ('future_items', True),
        'Item Count': ('item_count', False),
        'Step Back': ('_step_back', False),
        'Buffer Size': ('buffer_size', False)
        }

    row_list = [
        row_template.format(
            Label='',
            first_iter_item=label1, 
            second_iter_item=label2, 
            third_iter_item=label3)
                ]

    for label, attr_s in attr_group.items():
        requested_item, as_list = attr_s
        text_group = extract_attr_text(requested_item, iter1, iter2, iter3, as_list)
        text_line = row_template.format(Label=label, 
                        first_iter_item=text_group[1],
                        second_iter_item=text_group[2],
                        third_iter_item=text_group[3])
        row_list.append(text_line)
    
    iterator_compare_str = ''.join(row_list)
    
    return iterator_compare_str

### Initialize 2-line Section Tests

#### 2-line Section Source

In [5]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

## Iterator Options

### Two line section *StartSection*
- Start *Before* `StartSection`
- End *After* `EndSection`

In [6]:
two_line_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

#### Using a regular iterator

In [7]:
test_iter = iter(GENERIC_TEST_TEXT)
pprint(two_line_section.read(test_iter))

pprint(two_line_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
[]


![Good](../examples/Valid.png) A regular iterator is missing the buffer, so *Step Back* calls can loose future items.
- The second call to `pprint(two_line_section.read(test_iter))` returns an empty list because the second *StartSection* line is lost.
  
|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`[]`|`[]`|

#### Using a *BufferedIterator* source.

In [8]:
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(two_line_section.read(test_iter))
pprint(two_line_section.read(test_iter))
pprint(two_line_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
['StartSection Name: B', 'EndSection Name: B']
[]


![Good](../examples/Valid.png) Supplying a *BufferedIterator* preserves the *Step Back* item.
- The Third call to `pprint(two_line_section.read(test_iter))` returns an empty list because a third *StartSection* is not found.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|
|`[]`|`[]`|

# Simple sections experimenting with start and end settings

#### 2-line Section Source

In [9]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

## `end_on_first_item` Tests

The `end_on_first_item` parameter in a section definition determines whether the 
`end_section` break tests are applied to the first line in a section.
`end_on_first_item=True` does not *force* the section to complete after a single 
line.  It only makes it *possible* to stop after the first line.

The value of this parameter is that repeat sections do not require distinct
starting and ending sentinels. With `end_on_first_item=False` (the default) the 
same SectionBreak can be applied to both `start_section` and `end_section`.  
In this case, the second occurrence of the section begins immediately at the 
end of the previous section.

In [10]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]

#### Setting `end_on_first_item=False` (the default)
- Using identical `start_section` and `end_section`:
    > `start_section=SectionBreak('StartSection', break_offset='Before')`<br>
    > `end_section=SectionBreak('StartSection', break_offset='Before')`<br>

- Do not test first line of section (the default).
    > `end_on_first_item=False` 

In [11]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=False
    )

pprint(start_sub_section.read(GENERIC_TEST_TEXT))

['StartSection A', 'EndSection A']


![Good](../examples/Valid.png) The first section is returned as a list.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          ['StartSection A', 'EndSection A']
        </code></td>
        <td><code>
          ['StartSection A', 'EndSection A']
        </code></td></tr>
</table>

# Done To Here

In [12]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]
buffered_test_text = BufferedIterator(GENERIC_TEST_TEXT)


subsection = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=False
    )
#pprint(subsection.read(GENERIC_TEST_TEXT))

repeating_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('EndSection', break_offset='After'),
    processor=subsection
    )

pprint(repeating_section.read(GENERIC_TEST_TEXT))

[['StartSection A']]


In [13]:
print(buffered_iterator_compare(repeating_section.source, subsection.source, 
                                label1='top_source', 
                                label2='sub_section_source'))



	                    top_source                         sub_section_source                                                    
	Previous Items      ['Text to be ignored', 'StartSection A', 'EndSection A']['Text to be ignored', 'StartSection A', 'EndSection A']                                   
	Future Items        ['StartSection B']                 []                                                                    
	Item Count          3                                  3                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     



In [14]:
buffer_size = 10
num_items = 9

str_source = BufferedIterator((str(i) for i in range(num_items)), 
                              buffer_size=buffer_size)
pprint(list(str_source))


['0', '1', '2', '3', '4', '5', '6', '7', '8']


In [15]:
subsection = Section(
    section_name='SubSection',
    start_section=SectionBreak('3', break_offset='Before'),
    end_section=SectionBreak('5', break_offset='After'),
    end_on_first_item=False
    )
#pprint(subsection.read(GENERIC_TEST_TEXT))

repeating_section = Section(
    section_name='Top Section',
    #end_section=SectionBreak('7', break_offset='After'),
    processor=subsection
    )


In [16]:
def pairs(source):
    for item in source:
        yield tuple([item, next(source)])
        
        
def odd_nums(source):
    for item in source:
        if int(item)%2 == 1:
            yield item

In [17]:
str_source = BufferedIterator((str(i) for i in range(num_items)), 
                              buffer_size=buffer_size)
pprint(repeating_section.read(str_source))

[['3', '4', '5']]


In [18]:
print(buffered_iterator_compare(repeating_section.source, subsection.source, 
                                label1='top_source', 
                                label2='sub_section_source'))



	                    top_source                         sub_section_source                                                    
	Previous Items      ['4', '5', '6', '7', '8']          ['4', '5', '6', '7', '8']                                             
	Future Items        []                                 []                                                                    
	Item Count          9                                  9                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     



In [19]:
str_source = BufferedIterator((str(i) for i in range(num_items)), 
                              buffer_size=buffer_size)
pprint(subsection.read(str_source))
#pprint(subsection.read(str_source))

['3', '4', '5']


In [20]:
print(buffered_iterator_compare(subsection.source, str_source,
                                label1='sub_section_source', 
                                label2='str_source'))



	                    sub_section_source                 str_source                                                            
	Previous Items      ['2', '3', '4', '5']               ['0', '1', '2', '3', '4', '5']                                        
	Future Items        ['6']                              ['6']                                                                 
	Item Count          6                                  6                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  10                                                                    



In [21]:
print(subsection._source_index)
print(subsection.source.item_count)
print(subsection.source_item_count)
print(subsection.item_count)

[0, 4, 5, 6]
6
6
3


In [22]:
str_source = BufferedIterator((str(i) for i in range(num_items)), 
                              buffer_size=buffer_size)
subsection = Section(
    section_name='SubSection',
    start_section=SectionBreak('3', break_offset='Before'),
    end_section=SectionBreak('5', break_offset='After'),
    end_on_first_item=False
    )
pprint(subsection.read(str_source))

['3', '4', '5']


In [23]:
print(buffered_iterator_compare(subsection.source, str_source,
                                label1='sub_section_source', 
                                label2='str_source'))



	                    sub_section_source                 str_source                                                            
	Previous Items      ['2', '3', '4', '5']               ['0', '1', '2', '3', '4', '5']                                        
	Future Items        ['6']                              ['6']                                                                 
	Item Count          6                                  6                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  10                                                                    



In [24]:

print(f'Item Count: {repeating_section.item_count}  \t'
        f'Source Item Count {repeating_section.source.item_count}  \t'
        f'Section Source Index: {repeating_section.source_item_count}')
repeating_section.source_index



Item Count: 1  	Source Item Count 9  	Section Source Index: 9


[0, 7]

In [25]:
print(repeating_section._source_index)
print(repeating_section.source.item_count)
print(repeating_section.source.item_count)

[0, 7]
9
9


In [26]:

print(f'Item Count: {subsection.item_count}  \t'
        f'Source Item Count {subsection.source.item_count}  \t'
        f'Section Source Index: {subsection.source_item_count}')

subsection.source_index

Item Count: 3  	Source Item Count 6  	Section Source Index: 6


[0, 4, 5, 6]

#### Defining a top section with a repeating subsection
- Using identical `start_section` and `end_section`:
    > `start_section=SectionBreak('StartSection', break_offset='Before')`<br>
    > `end_section=SectionBreak('StartSection', break_offset='Before')`<br>

- Do not test first line of section (the default).
    > `end_on_first_item=False` 

In [27]:
buffered_test_text = BufferedIterator(GENERIC_TEST_TEXT)

start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=False
    )

repeating_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('More text to be ignored', break_offset='Before'),
    processor=start_sub_section
    )
pprint(repeating_section.read(buffered_test_text))

[['StartSection A', 'EndSection A'], ['StartSection B', 'EndSection B']]


![Error](../examples/Error.png) Source is not resetting to the correct spot.

In [28]:
print(buffered_iterator_compare(buffered_test_text, start_sub_section.source,
                                label1='Top Source', label2='Subsection Source'))

	                    Top Source                         Subsection Source                                                     
	Previous Items      ['StartSection A', 'EndSection A', 'StartSection B', 'EndSection B']['Text to be ignored', 'StartSection A', 'EndSection A', 'StartSection B', 'EndSection B']                                   
	Future Items        ['More text to be ignored']        []                                                                    
	Item Count          5                                  5                                                                     
	Step Back           0                                  0                                                                     
	Buffer Size         5                                  5                                                                     



![Error](../examples/Error.png) Source is not resetting to the correct spot.
<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
        Top Source<br>
        Previous Items: ['Text to be ignored', 'StartSection A', 'EndSection A', 'StartSection B', 'EndSection B']<br>
        Future Items: ['More text to be ignored']<br>
        Item Count: 5
        </code></td>
        <td><code>
        Top Source<br>
        Previous Items: ['StartSection A']<br>
        Future Items: ['EndSection A', 'StartSection B', 'EndSection B', 'More text to be ignored']<br>
        Item Count: 2
        </code></td></tr>
</table>

In [29]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=False
    )

repeating_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('More text to be ignored', break_offset='Before'),
    processor=start_sub_section
    )
pprint(repeating_section.read(GENERIC_TEST_TEXT))

[['StartSection A', 'EndSection A'], ['StartSection B', 'EndSection B']]


![Error](../examples/Error.png) Only the first subsection is returned.
<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
        ['StartSection A', 'EndSection A']
        </code></td>
        <td><code>
          [<br>
          ['StartSection A', 'EndSection A'],<br>
           ['StartSection B', 'EndSection B']<br>
          ]
        </code></td></tr>
</table>

![Good](../examples/Valid.png) Both subsections are returned as a list of lists.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
          ['StartSection A', 'EndSection A'],<br>
           ['StartSection B', 'EndSection B']<br>
          ]
        </code></td>
        <td><code>
          [<br>
          ['StartSection A', 'EndSection A'],<br>
           ['StartSection B', 'EndSection B']<br>
          ]
        </code></td></tr>
</table>

- Using the same section definition, except allow testing of the first line of 
  the section.
    > `end_on_first_item=True` 

In [30]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True
    )

pprint(start_sub_section.read(GENERIC_TEST_TEXT))

[]


![Good](../examples/Valid.png) The ending SectionBreak triggers on the same 
item that triggers the start of the section.  This will always result in an 
empty section.

|Expected|Actual|
|-|-|
|[]|[]|

#### Single Line Section.
- Using the same sentinel *('EndSection')*, but different `break_offset`.
    > `start_section=SectionBreak('EndSection', break_offset='Before')`<br>
    > `end_section=SectionBreak('EndSection', break_offset='After')`<br>

- Allow testing of the first line of section.
    > `end_on_first_item=True` 

In [31]:
end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True
    )

pprint(end_sub_section.read(GENERIC_TEST_TEXT))

['EndSection A']


![Good](../examples/Valid.png) Single line section.
- Starts *Before* **EndSection** 
- Ends *After* **EndSection** (the same line)

|Expected|Actual|
|-|-|
|['EndSection A']|['EndSection A']|

#### Same section definition as above except with `end_on_first_item=False` 

In [32]:
end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=False
    )

pprint(end_sub_section.read(GENERIC_TEST_TEXT))

['EndSection A', 'StartSection B', 'EndSection B']


![Good](../examples/Valid.png) Section continues until **After** next 
*EndSection* is found

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          ['EndSection Name: A',<br> 
          'StartSection Name: B',<br>
          'EndSection Name: B']
        </code></td>
        <td><code>
          ['EndSection Name: A',<br>
          'StartSection Name: B',<br>
          'EndSection Name: B']
        </code></td></tr>
</table>

## Multiple Single line sections
### Single line section *StartSection*
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

In [33]:
sub_section1 = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))

['StartSection A']
['StartSection B']
[]


![Good](../examples/Valid.png) Single line `StartSections` are returned
|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['StartSection Name: A']`|
|`['StartSection Name: B']`|`['StartSection Name: B']`|
|`[]`|`[]`|

### Defining a section that calls `sub_section1`

In [34]:
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[sub_section1]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B']


![Good](../examples/Valid.png) Both *StartSection* lines are found.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
        [<br>
          ['StartSection Name: A'],<br>
          ['StartSection Name: B'],<br>
        ]</code></td>
        <td><code>
        [<br>
          ['StartSection Name: A'],<br>
          ['StartSection Name: B'],<br>
        ]</code></td></tr>
</table>

### Multiple Attempts to define Single line *EndSection* 
#### Attempt #1. Copy *StartSection* design
> - Start *Before* `EndSection`
> - End *Before* `StartSection`


In [35]:
sub_section2a = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))

['EndSection A']
['EndSection B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) The result is expected, but not desired.
- This doesn't work well because there isn't another *StartSection* after the last it *EndSection*.
- As a result, it includes the unwanted text after the last *EndSection*.
 
|Expected|Actual|
|-|-|
|`['EndSection A']`|`['EndSection A']`|
|`['EndSection B', 'More text to be ignored']`|`['EndSection B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #2
> - Start *Before* `EndSection`
> - End *After* `EndSection`

- This doesn't work because The first line is not tested so it only breaks after the second occurrence of *EndSection*.

In [36]:
sub_section2b = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))

['EndSection A', 'StartSection B', 'EndSection B']
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection A', 'StartSection B', 'EndSection B']`|`['EndSection A', 'StartSection B', 'EndSection B']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #3
> - Start *Before* `EndSection`
> - End *After* `EndSection`
> - Enable testing of first item

- This should work because the first line should be tested and trigger a break.

In [37]:
sub_section2c = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))

['EndSection A']
['EndSection B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection A']`|`['EndSection A']`|
|`['EndSection B']`|`['EndSection B']`|
|`[]`|`[]`|


#### Attempt #3.5
> - Start *Before* `EndSection`
> - End *Before* `EndSection`
> - Enable testing of first item

- This should __<u>not</u>__ work because the first line should be tested and 
  trigger a break before returning anything.

In [38]:
sub_section2cc = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))

[]
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`[]`|`[]`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #4
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Enable testing of first item

- This should work because the first line should be tested and always trigger a break.

In [39]:
sub_section2d = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))

['EndSection A']
['EndSection B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['StartSection A']`|`['EndSection A']`|
|`['StartSection B']`|`['EndSection B']`|
|`[]`|`[]`|

#### Attempt #5
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger on the second line.

In [40]:
sub_section2e = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))

['EndSection A', 'StartSection B']
['EndSection B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection A', 'StartSection B']`|`['EndSection A', 'StartSection B']`|
|`['EndSection B', 'More text to be ignored']`|`['EndSection B', 'More text to be ignored']`|
|`[]`|`[]`|

**`end_section=SectionBreak(True)` defaults to `break_offset='Before'`**

#### Attempt #6
> - Start *Before* `EndSection`
> - End *After* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger *After* the second line.

In [41]:
sub_section2f = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))

['EndSection A', 'StartSection B']
['EndSection B', 'More text to be ignored']
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection A', 'StartSection B']`|`['EndSection A', 'StartSection B']`|
|`['EndSection B', 'More text to be ignored']`|`['EndSection B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #7
> - Start *Before* `EndSection`
> - Enable testing of first item
> - No End setting

- This should start with the first *EndSection* line and not stop until the end of the iterator.

In [42]:
sub_section2g = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))

['EndSection A', 'StartSection B', 'EndSection B', 'More text to be ignored']
[]
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Single line section *EndSection* __Final Design__.
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should work because the break should trigger *Before* the second line.

In [43]:
sub_section2 = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))

['EndSection A']
['EndSection B']
[]


![Good](../examples/Valid.png) End breaks *Before* second line. 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

### Combined Start and End subsections Single line section 
***sub_section1***
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

***sub_section2*** 
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

`subsections=[sub_section1, sub_section2]`

In [44]:

full_section = Section(
    section_name='Full',
    subsections=[sub_section1, sub_section2]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B',
 'More text to be ignored']


![Good](../examples/Valid.png) Tree list levels with single item in deepest level andm two items in each of the other two levels.

|Expected|Actual|
|-|-|
|`[['StartSection Name: A'], ['EndSection Name: A']]`|`[['StartSection Name: A'], ['EndSection Name: A']]`|
|`[['StartSection Name: B'], ['EndSection Name: B']]`|`[['StartSection Name: B'], ['EndSection Name: B']]`|

## `keep_partial` Tests
**keep_partial is not currently being implemented.**
 
The `keep_partial` parameter in a section definition only applies when a list of subsections is given.  For all other cases the `keep_partial` parameter is ignored.
With a parent section definition that contains a list of subsections, a situation can occur where the parent section or the source ends before all of the subsections have been completed. 
- When `keep_partial=True` is set the final subsection list returned will be missing items for the subsections that did nopt get read.

- When `keep_partial=False` (the default) is set the incomplete final subsection list is not returned.

### Defining a Top Section that calls two single line subsections.

#### Two Subsection definitions 

In [45]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Defining ***top_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

In [46]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section]
    )
pprint(top_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B']


![Good](../examples/Valid.png) The Subsections are returned as lists of lists.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection Name: A'],<br>
             ['EndSection Name: A']],<br>
             [['StartSection Name: B'],<br>
              ['EndSection Name: B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection Name: A'],<br>
             ['EndSection Name: A']],<br>
             [['StartSection Name: B'],<br>
              ['EndSection Name: B']]<br>
          ]
        </code></td></tr>
</table>

#### Including unwanted text in between the start and end of subsection C

In [47]:
GENERIC_TEST_TEXT1 = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'Even more text to be ignored', 
    ]

pprint(top_section.read(GENERIC_TEST_TEXT1))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B',
 'StartSection C']


![Good](../examples/Valid.png) Subsections A and B are returned Subsections C 
has an empty list instead of 'EndSection C'.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
             [['StartSection B'], ['EndSection B']]<br>
             [['StartSection C'], []]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C'], []]<br>            
          ]
        </code></td></tr>
</table>

#### Setting `keep_partial=False`
- This should be the default

In [48]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=False
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B',
 'StartSection C']


![Good](../examples/Valid.png) Subsections A and B are returned Subsections C 
has an empty list instead of 'EndSection C'.
- 3rd section group never starts because **ignored** *Top Section* break line 
  occurs before next **EndSection**, so 2nd never finishes.


<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
             [['StartSection B'], ['EndSection B']]<br>
             [['StartSection C'], []]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C'], []]<br>            
          ]
        </code></td></tr>
</table>

#### Setting `keep_partial=True`

In [49]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B',
 'StartSection C']


![Good](../examples/Valid.png) Subsections A and B are returned Subsections C 
has an empty list instead of 'EndSection C'.
- 3rd section group never starts because **ignored** *Top Section* break line 
  occurs before next **EndSection**, so 2nd never finishes.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C'], []]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C'], []]<br>
          ]
        </code></td></tr>
</table>

##### Using same `keep_partial=True` setting with original source.

In [50]:
pprint(top_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B']


![Good](../examples/Valid.png) `keep_partial=True` has no effect because both 
subsections A and B are completed. 

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection Name: A'], ['EndSection Name: A']],<br>
            [['StartSection Name: B'], ['EndSection Name: B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection Name: A'], ['EndSection Name: A']],<br>
            [['StartSection Name: B'], ['EndSection Name: B']]<br>
          ]
        </code></td></tr>
</table>

##### Using same `keep_partial=True` with only `end_sub_section`.

In [51]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B',
 'StartSection C']


![Good](../examples/Valid.png) `keep_partial=True` has no effect because single 
subsections are processed differently. 
- Results in one less level of lists.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            ['EndSection Name: A'],<br>
            ['EndSection Name: B']<br>
          ]
        </code></td>
        <td><code>
          [<br>
            ['EndSection Name: A'],<br>
            ['EndSection Name: B']<br>
          ]
        </code></td></tr>
</table>

#### Dropping the corresponding *EndSection* for a *StartSection*.

In [52]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',  # Missing 'EndSection B',
    
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'Even more text to be ignored', 
    ]

In [53]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'StartSection C']


![Good](../examples/Valid.png) Expecting `['StartSection B'], []`.
- 3rd section group never starts because **ignored** *Top Section* break line 
  occurs before next **EndSection**, so 2nd section never finishes.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], []]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], []]<br>
          ]
        </code></td></tr>
</table>

In [54]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=False
    )

pprint(top_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'StartSection C']


![Good](../examples/Valid.png) Expecting partial `['StartSection B'], []`.
- 2nd section group never finishes because **ignored** *Top Section* break 
  line occurs.  Partial subsection should have been dropped.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']]<br>
            [['StartSection B'], []]<br> 
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], []]<br>
          ]
        </code></td></tr>
</table>

### Including unwanted text in between subsections

In [55]:
GENERIC_TEST_TEXT1 = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'More text to be ignored', 
    'EndSection Name: B',
    'Even more text to be ignored', 
    ]

pprint(full_section.read(GENERIC_TEST_TEXT1))

['Text to be ignored',
 'StartSection Name: A',
 'EndSection Name: A',
 'StartSection Name: B',
 'More text to be ignored',
 'EndSection Name: B',
 'Even more text to be ignored']


![Good](../examples/Valid.png) The *to be ignored* text between subsections is dropped.

|Expected|Actual|
|-|-|
|`[['StartSection Name: A'], ['EndSection Name: A']]`|`[['StartSection Name: A'], ['EndSection Name: A']]`|
|`[['StartSection Name: B'], ['EndSection Name: B']]`|`[['StartSection Name: B'], ['EndSection Name: B']]`|

## Hysteresis Bug Fixed  ![Good](../examples/Valid.png) 
When `full_section` contains 
`end_section=SectionBreak('ignored', break_offset='Before')`, repeated calls produce different effects.

#### Define `SubSection`: 
- Start *Before* `StartSection`
- End *After* `EndSection`

In [56]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before', name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', name='SubSectionEnd')
    )

1. Clear Hysteresis by running `full_section` without setting `end_section`
2. Define `full_section` with 
`end_section=SectionBreak('ignored', break_offset='Before')`.

In [57]:
full_section = Section(
    section_name='Full',
    subsections=sub_section,
    keep_partial=False
    )
a = full_section.read(GENERIC_TEST_TEXT)
b = full_section.read(GENERIC_TEST_TEXT)

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B']


![Good](../examples/Valid.png) 
First time it runs as expected
- Both full subsections found. No empty lists afterwards.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|

In [58]:
pprint(full_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection A',
 'EndSection A',
 'StartSection B',
 'EndSection B']


![Good](../examples/Valid.png)  Hysteresis Fixed
- Before bug fix, The second time, a line in the first subsection is missing.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


## Check *Source* status
- Check ___Source___ status after section read.

In [59]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before', name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', name='SubSectionEnd')
    )
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )
a = full_section.read(GENERIC_TEST_TEXT)
print('previous_items')
pprint(list(full_section.source.previous_items))
print()
print('future_items')
pprint(list(full_section.source.future_items ))

previous_items
['StartSection A', 'EndSection A', 'StartSection B', 'EndSection B']

future_items
['More text to be ignored']


#### Expected source values
<table>
<thead>
  <tr>
    <th>Buffer</th>
    <th>Expected</th>
    <th>Actual</th>
    <th>Test</th>
  </tr>
</thead>
<tr><td><b>previous_items</b></td>
<td><code>['StartSection A', 'EndSection A',<br>
           'StartSection B', 'EndSection B']</code></td>
<td><code>['StartSection A', 'EndSection A',<br>
           'StartSection B', 'EndSection B']</code></td>
<td><img src="../examples/Valid.png" alt="Good"/></td></tr>
<tr><td><b>future_items</b></td>
<td><code>['More text to be ignored']</code></td>
<td><code>['More text to be ignored']</code></td>
<td><img src="../examples/Valid.png" alt="Good"/></td></tr>
</table>

## Check Context

***Context Standard Items***
<table>
<thead><th>Item Key</th><th>Value Description</th></thead>
<tr><td>Current Section</td>
<td>The assigned name of the section <i>self.section_name</i></td></tr>

<tr><td>Skipped Lines</td>
<td>A list of items in the source occurring before the first item of the section</td></tr>

<tr><td>Status</td>
<td>The current section scanning status one of:<ul>
    <li>Not Started
    <li>At section start
    <li>Scan In Progress
    <li>Break Triggered
    <li>Scan Complete
    <li>End of Source
    </ul></td></tr>
<tr><td>Break</td>
<td>The name assigned to the SectionBreak that most recently triggered. 
Before the section start this is not defined. 
While the section scan is in progress, this is the name of the StartSection. 
After the section ends, this is the name of the the EndSection.</td></tr>
<tr><td>
Event</td>
<td>The result from the SectionBreak that most recently triggered; the 
<i>TriggerEvent.test_value</i>.  
For more information see the TriggerEvent documentation.</td></tr>
</table>

- Compare `full_section` and `sub_section` ___Context___ after reading.


In [60]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'Text between sections',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

In [61]:
sub_section = Section(
    section_name='Sub-Section',
    start_section=SectionBreak('StartSection', break_offset='Before',
                               name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', 
                             name='SubSectionEnd')
    )
full_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before', 
                             name='End Section'),
    subsections=sub_section
    )
a = full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'})
print()
print('full_section context')
pprint(full_section.context)
pprint(full_section.scan_status)
print()
print('sub_section context')
pprint(sub_section.context)
pprint(sub_section.scan_status)


full_section context
{'Break': 'End Section',
 'Current Section': 'Top Section',
 'Dummy': 'Blank1',
 'Event': 'ignored',
 'Skipped Lines': [],
 'Status': 'Break Triggered'}
'Break Triggered'

sub_section context
{'Current Section': 'Sub-Section', 'Status': 'Not Started'}
'Not Started'


**Context Tests**
<style type="text/css">
.tg th{font-weight:bold;text-align:center;vertical-align:middle}
.tg td{font-weight:normal;text-align:left;vertical-align:top}
.rh td{font-weight:bold;text-align:center;vertical-align:middle}
</style>

<table class="tg">
    <thead>
        <tr>
            <th rowspan="2" text-align="center">Key</th>
            <th colspan="3">Top Section</th>
            <th colspan="3">Sub-Section</th>
        </tr><tr>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
        </tr>
    </thead>
        <tr><td class="rh">Current Section</td>
            <td>'Top Section'</td>
            <td>'Top Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
            <td>'Sub-Section'</td>
            <td>'Sub-Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
        </tr>
    <tr><td class="rh">Dummy</td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Skipped Lines</td>  
        <td>[]</td>
        <td>[]</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>['Text between sections']</td>
        <td>['Text between sections']</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Status</td>  
        <td>'Break Triggered'</td>
        <td>'Break Triggered'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'End of Source'</td>
        <td>'End of Source'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Break</td>  
        <td>'End Section'</td>
        <td>'End Section'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'SubSectionStart'</td>
        <td>'SubSectionStart'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Event</td>  
        <td>'ignored'</td>
        <td>'ignored'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'StartSection'</td>
        <td>'StartSection'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
</table>

- ***Current Section*** is the name of the section given at the section definition.
    - For _full_section_,  `name='End Section'`
    - For _sub_section_,   `name='Sub-Section'`
- ***Dummy*** is a context element supplied at the time `full_section.read()` was 
  called. Since it is not one of the standard context items it is just along for 
  the ride unless a user defined function explicitly modifies it. 
    - The initially supplied value of ***Dummy*** = _'Blank1'_ is the same for 
      both _full_section_ and _sub_section_.
- ***Skipped Lines*** is the list of unused lines encountered before the beginning 
  of the section.
    - For _full_section_, start_section is not given so ***Skipped Lines*** is 
      an empty list.
    - For _sub_section_, ***Skipped Lines*** is the lines skipped before the 
      most recent sub-section: ['Text between sections']
- ***Status*** is the section state. 
    - _full_section_ has ended because the `end_section` break has triggered, so
      ***Status*** = 'Break Triggered'
    - _sub_section_ receives it's source from _full_section_, but 
      _full_section_ has ended.  _sub_section_ interprets this as and end to 
      it's source, so ***Status*** = 'End of Source'
- ***Break*** is the name of the most recent break triggered while the section is 
   active.
    - The `end_section` break of _full_section_ has triggered.  The name of 
      this SectionBreak was given as 'End Section', so 
      ***Break*** = 'End Section'.
    - The `end_section` break of _sub_section_ never triggered because it's 
      source ended first. The most recent section break for _sub_section_ was 
      the `start_section`, so ***Break*** = 'SubSectionStart'.
- ***Event*** is the Trigger specific value resulting from the last break.  For 
  string based breaks, ***Event*** is the string used to test for the break.
    - The `end_section` break of _full_section_ was triggered by the string 
      'ignored', so ***Event*** = 'ignored'.
    - The `start_section` break of _sub_section_ was triggered by the string 
      'StartSection', so ***Event*** = 'StartSection'.

**********

# Section Break options

## Three line sections

In [62]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

**********

### Initial Section and Sub-Section Definitions

- Only definition line is:<br>
`subsections=[sub_section]`

In [63]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True
    subsections=sub_section  
    )
pprint(full_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) Results in all lines in one sub-list

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['Text to be ignored',<br>
       'StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
      ]</code></td>
  <td><code>
    [<br>
      ['Text to be ignored',<br>
       'StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
      ]</code></td>
</table>



### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [64]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True,
    subsections=sub_section
    )
pprint(full_section.read(GENERIC_TEST_TEXT2))

['StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A',<br>
       'Unwanted text between sections',<br>
       'StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B',<br>
       'StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C',<br>
       'Even more text to be ignored']<br>
    ]</code></td>
</table>



### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )
```

In [65]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_on_first_item=True,    
    subsections=sub_section
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

['StartSection A', 'MiddleSection A', 'EndSection A']
['StartSection B', 'MiddleSection B', 'EndSection B']
['StartSection C', 'MiddleSection C', 'EndSection C']
[]


- ![Good](../examples/Valid.png) Includes all three lines of first section in single sub-list.
- skips the *'Unwanted text between sections'* line.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']<br>
    ]</code></td>
</tr>
<tr>
  <td><code>
    [<br>
      ['StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection B',<br>
       'MiddleSection B',<br>
       'EndSection B']<br>
    ]</code></td>
</tr>
<tr>
  <td><code>
    [<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
</tr>
<tr><td><code>[]</code></td><td><code>[]</code></td></tr>
</table>



### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(
    section_name='Multi',
    subsections=full_section 
    )
```

In [66]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(
    section_name='Multi',
    subsections=full_section
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png)Includes all three lines of each section in its own sub-list.
- Skips the *'Unwanted text between sections'* line.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td>    
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **Before** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=sub_section 
    )

multi_section = Section(
    section_name='Multi',
    subsections=full_section 
    )
```

In [67]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) Includes *'Unwanted text between sections'* and
  *'Even more text to be ignored'*.
- Includes *'Unwanted text between sections'* because the end of section A is 
  triggered by *'StartSection B'*. 
- Includes *'Even more text to be ignored'* because there are no more 
  *'StartSection'* lines to trigger an `end_section` break.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A',<br>
        'Unwanted text between sections']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
        'MiddleSection C',<br>
        'EndSection C',<br>
        'Even more text to be ignored']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A',<br>
        'Unwanted text between sections']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
        'MiddleSection C',<br>
        'EndSection C',<br>
        'Even more text to be ignored']]<br>
    ]</code></td>
  </tr>
</table>

### Add `end_on_first_item=True` to Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    subsections=sub_section 
    )
```

In [68]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    subsections=sub_section
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

test_iter = iter(GENERIC_TEST_TEXT2)
pprint(full_section.read(test_iter))
pprint(full_section.read(test_iter))

[]
[]
[]
[]


- ![Good](../examples/Valid.png) Results in empty list because it both starts 
  and ends on first *'StartSection'*.

|Expected|Actual|
|-|-|
|[]|[]|

### <span style="color: red; font-weight: Bold">Endless loop with `multi_section`</span>

<span style="color: red; font-weight: Bold; font-variant: small-caps">
Do Not Try This
</span>

- Use `multi_section` to read multiple empty sections
```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True,
    subsections=sub_section 
    )
multi_section = Section(section_name='Multi',
    subsections=full_section
    )
print(multi_section.read(GENERIC_TEST_TEXT2))
```
- `full_section` both starts and ends on first *'StartSection'*.
- No progress is made in in iterating through the source.
- Trying to read multiple sections will result in an endless loop.

**********

## SubSection Break Options
- Section break checks start with the top most section
- The section level containing the break will affect the way the list are nested.

### Add `start_section` to *sub_section* and remove it from *full_section*

- SubSection start **Before** *StartSection*
- Section end **After** *EndSection*


```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [69]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

In [70]:
pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) behaves the same way as when `start_section` 
 was defined in *full_section*.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td>    
  <td><code>
    [<br>
      [['StartSection A',<br>
        'MiddleSection A',<br>
        'EndSection A']],<br>
      [['StartSection Name: B',<br>
        'MiddleSection B',<br>
        'EndSection Name: B']]<br>
      [['StartSection Name: C',<br>
        'MiddleSection C',<br>
        'EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

### Add `end_section` to *sub_section* and remove it from *full_section*

- SubSection end **After** *EndSection*
- Section start **Before** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before')
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [71]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) Includes *'Unwanted text between sections'* and
  *'Even more text to be ignored'*.
- `sub_section` has no `start_section` defined, so the it starts immediately. 
- `full_section` never ends and so the second sub-section starts right after 
  the first one ends. 
- **Note:** 
    - sub-list nesting is different:
        > `[[[Section Lines], [Next Section Lines], ... ]]`<br>
        > VS.<br>
        > `[[[Section Lines]], [[Next Section Lines]], ... ]`<br>
    - *'Even more text to be ignored'* is in its own sub-section.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['Unwanted text between sections',<br>
       'StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C'],<br>
      ['Even more text to be ignored']<br>
    ]]</code></td>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['Unwanted text between sections',<br>
       'StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C'],<br>
      ['Even more text to be ignored']<br>
    ]]</code></td>
  </tr>
</table>

### Add both `start_section` and `end_section` to *sub_section*. No breaks defined in *full_section*

- SubSection start **Before** *StartSection*
- SubSection end **After** *EndSection*


```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [72]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) similar results to having both breaks defined in *full_section*.
- **Note:** sub-list nesting is different:
    > `[[[Section Lines], [Next Section Lines], ... ]]`<br>
    > VS.<br>
    > `[[[Section Lines]], [[Next Section Lines]], ... ]`<br>

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]]</code></td>
  <td><code>
    [[<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]]</code></td>
  </tr>
</table>

### Read from *full_section* instead of *multi_section*

In [73]:
pprint(full_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) Similar results to reading *multi_section*.

- **Note:** One less list level.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  <td><code>
    [<br>
      ['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A'],<br>
      ['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B'],<br>
      ['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']<br>
    ]</code></td>
  </tr>
</table>

### Add `start_section` and `end_section` to both *sub_section* and *full_section*.

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *StartSection*
- SubSection end **After** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section
    )
```

In [74]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    #end_section=SectionBreak(True)
    #end_on_first_item=True
    )
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section,
    #end_on_first_item=True
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) behaves the same way as when *sub_section* 
  did not have any breaks defined (including list levels).

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['StartSection A',<br>
       'MiddleSection A',<br>
       'EndSection A']],<br>
      [['StartSection B',<br>
        'MiddleSection B',<br>
        'EndSection B']],<br>
      [['StartSection C',<br>
       'MiddleSection C',<br>
       'EndSection C']]<br>
    ]</code></td>
  </tr>
</table>

### Change Start Sub-Section to **After**

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **After** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [75]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) *StartSection* lines are dropped.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr>
  <td><code>
    [<br>
      [['MiddleSection A',
       'EndSection A']],<br>
      [['MiddleSection B',
        'EndSection B']],<br>
      [['MiddleSection C',
       'EndSection C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['MiddleSection A',
       'EndSection A']],<br>
      [['MiddleSection B',
        'EndSection B']],<br>
      [['MiddleSection C',
       'EndSection C']]<br>
    ]</code></td>
  </tr>
</table>

### Run with two-line sections:
> `[`<br>
> `'Text to be ignored',`<br>
> `'StartSection Name: A',`<br>
> `'EndSection Name: A',`<br>
> `'Text between sections',`<br>
> `'StartSection Name: B',`<br>
> `'EndSection Name: B',`<br>
> `'More text to be ignored'`<br>
> `]`<br>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **After** *StartSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [76]:
pprint(multi_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection Name: A',
 'EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']


- ![Good](../examples/Valid.png) Results in both 'EndSections'
<table><thead><th>Expected</th><th>Actual</th></thead>
 <tr> <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
    ]</code></td></tr>
</table>

In [77]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=sub_section 
    )

multi_section = Section(section_name='Multi',
    subsections=full_section,
    #end_on_first_item=True
    )

pprint(multi_section.read(GENERIC_TEST_TEXT2))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C',
 'Even more text to be ignored']


- ![Good](../examples/Valid.png) Results in three 'EndSections'
<table><thead><th>Expected</th><th>Actual</th></thead>
 <tr> <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
      [['EndSection Name: C']]<br>
    ]</code></td>
  <td><code>
    [<br>
      [['EndSection Name: A']],<br>
      [['EndSection Name: B']]<br>
      [['EndSection Name: C']]<br>
    ]</code></td></tr>
</table>

In [78]:
GENERIC_TEST_TEXT

['Text to be ignored',
 'StartSection Name: A',
 'EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']

In [79]:
pprint(full_section.read(GENERIC_TEST_TEXT))

['StartSection Name: A', 'EndSection Name: A']


- ![Good](../examples/Valid.png) Results in first 'EndSection'
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [80]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['Text to be ignored',
 'StartSection Name: A',
 'EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']


- ![Good](../examples/Valid.png) Results in everything starting with first 'EndSection'.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td>
<td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [81]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']


- ![Good](../examples/Valid.png) Results in one line section starting with first 'EndSection'.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<BR>
  'StartSection Name: B',
  'EndSection Name: B',<BR>
  'More text to be ignored']]
  </code></td>
<td><code>
[['EndSection Name: A',<BR>
  'StartSection Name: B',
  'EndSection Name: B',<BR>
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and <br>*Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [82]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']


- ![Good](../examples/Valid.png) Two competing `start_section`s Results in one line section starting with first 'EndSection'.
- `full_section`'s `start_section=SectionBreak('StartSection', break_offset='After')` drives the start.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td>
<td><code>
  [['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and <br>*Start Before End Section* to SubSection Definition and <br>set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [83]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']


- ![Good](../examples/Valid.png) `end_on_first_item=True,` has no effect because no end_section is given.
- Two competing `start_section`s Results in one line section starting with first 'EndSection'.
- `full_section`'s `start_section=SectionBreak('StartSection', break_offset='After')` drives the start.
- There is no `end_section`, so everything after `start_section` triggers will be read.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td>
<td><code>
  [['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]</code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition, and <br>for SubSection Definition, set *Start* to  *Before EndSection*<br> and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [84]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']


- ![Good](../examples/Valid.png) Results in two one-line sections.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td>
<td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td></tr>
</table>

### Add *Start After StartSection* and <br>*End Before EndSection* to Section Definition, and <br>for SubSection Definition, set <br>*Start* to  *Before EndSection* and <br>*End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),`<br>
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`
    

In [85]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B']


- ![Good](../examples/Valid.png) Results in two one-line sections.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td>
<td><code>
[<br>
['EndSection Name: A'],<br>
['EndSection Name: B']<br>
]
  </code></td></tr>
</table>

<b>Break Analysis</b>
<table><thead>
<th>Text Line</th><th>Full Section</th><th>SubSection</th>
</thead>
<tr><td>'Text to be ignored'</td><td>Skipped</td><td>Skipped</td></tr>
<tr><td>'StartSection Name: A'</td><td>Start Section After</td><td>Skipped</td></tr>
<tr><td><b>'EndSection Name: A'</b></td><td><u>Processed</u><br>No End Section<br>(<code>end_on_first_item=False</code>)</td><td>Start Section Before<br>(<u>Processed</u>)<br>No End Section<br>(<code>end_on_first_item=False</code>)</td></tr>
<tr><td>'Text between sections'</td><td><i>Processed</i></td><td>End Section Before</td></tr>
<tr><td>'StartSection Name: B'</td><td><i>Processed</i></td><td>Skipped</td></tr>
<tr><td><b>'EndSection Name: B'</b></td><td><u>Processed</u><br>End Section After</td><td>Start Section Before<br>(<u>Processed</u>)<br>No End Section<br>(<code>end_on_first_item=False</code>)</td></tr>
<tr><td>'More text to be ignored'</td><td>End of Section</td><td>Not Reached</td></tr>
 </table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`
    

In [86]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['StartSection Name: A', 'EndSection Name: A']


- ![Good](../examples/Valid.png) Results in single line section.
- 'EndSection Name: A' is now the second line in Full Section, so the section ends after this line.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>for SubSection Definition, set <br>*Start* to *Before EndSection*, <br>*End* to *`True` (Always Break)* and <br>*Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),` <br>   
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [87]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['StartSection Name: A', 'EndSection Name: A']


- ![Good](../examples/Valid.png) Results in the same single line section.
- `keep_partial=True` has no effect.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`<br>
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [88]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['StartSection Name: A', 'EndSection Name: A']


- ![Good](../examples/Valid.png) Results in Two line section
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[[<br>'StartSection Name: A',<br>'EndSection Name: A'<br>]]</code></td>
<td><code>[[<br>'StartSection Name: A',<br>'EndSection Name: A'<br>]]</code></td></tr>
</table>

### Add *Start __After__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`<br> 
> `end_section=SectionBreak('EndSection', break_offset='After'),` <br>    
    

In [89]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A',
 'Text between sections',
 'StartSection Name: B',
 'EndSection Name: B']


- ![Good](../examples/Valid.png) Starts with first 'EndSection' ends with second 'EndSection'.
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',<br>
  'EndSection Name: B']]
</code></td>
<td><code>
[['EndSection Name: A',<br>
  'Text between sections',<br>
  'StartSection Name: B',<br>
  'EndSection Name: B']]
</code></td></tr>
</table>

### Set `end_on_first` to *True*. <br>*Start __After__ StartSection* and <br>*End Before EndSection* to Section Definition, and <br>don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`<br> 
> `end_section=SectionBreak('EndSection', break_offset='After'),`<br>
> `end_on_first_item=True`<br>

In [90]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    #subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

['EndSection Name: A']


- ![Good](../examples/Valid.png) Results in one line section
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

## Three line sections

In [91]:
GENERIC_TEST_TEXT3 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'Random text in the middle of a section',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

#### Single line subsection definitions 

In [92]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

middle_sub_section = Section(
    section_name='MiddleSubSection',
    start_section=SectionBreak('MiddleSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Defining ***three_part_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

- Contains 3 subsections:
    > `[start_sub_section, middle_sub_section, end_sub_section]`

In [93]:
three_part_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, middle_sub_section, end_sub_section]
    )
pprint(three_part_section.read(GENERIC_TEST_TEXT3))

['Text to be ignored',
 'StartSection A',
 'MiddleSection A',
 'EndSection A',
 'Unwanted text between sections',
 'StartSection B',
 'Random text in the middle of a section',
 'MiddleSection B',
 'EndSection B',
 'StartSection C',
 'MiddleSection C',
 'EndSection C']


![Good](../examples/Valid.png) All 3 sections and subsections are completed. 

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td></tr>
</table>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [94]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT3))

['StartSection A', 'MiddleSection A', 'EndSection A']


![Good](../examples/Valid.png) First section and all 3 subsections are completed. 

<table>
  <thead><th>Expected</th><th>Actual</th></thead>
  <tr><td><code>
    [[<br>
    'StartSection Name: A',<br> 
    'MiddleSection A',<br> 
    'EndSection Name: A'<br>
    ]]
    </code></td>
    <td><code>
    [[<br>
    'StartSection Name: A',<br> 
    'MiddleSection A',<br> 
    'EndSection Name: A'<br>
    ]]
  </code></td></tr>
</table>